<a href="https://colab.research.google.com/github/poorvitiwary/Computer_vision_Age/blob/main/training_test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
import numpy as np
print(tf.__version__)

1.15.2


In [ ]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
from sklearn.preprocessing import OneHotEncoder,LabelEncoder # Label encoding, 1-hot encoding, multi-encoding
# LABEL binarizer is a 1-hot encoded MATRIX 
import cv2
import numpy as np
import random
import imutils
from imutils import paths
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import backend
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.keras.layers import  GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

In [ ]:
!ls data 

 children  'female adult'  'male adult'


In [ ]:
HP_dataset = 'data'
HP_model_path = 'bin/model'
HP_binarized_labels = 'bin/labels'
HP_metrics_storage = 'eval'
HP_test_dataset = 'test'
HP_epoch = 100
HP_init_lr = 1e-3 # learning_rate = 0.001
HP_batch_size = 32
HP_image_dim = (96,96,3)

In [ ]:
data = []
labels = [] 
# read all images
all_images = sorted(list(paths.list_images(HP_dataset)))
all_images[:5]

['data/children/13_0_0_20170110225717809.jpg',
 'data/children/13_0_0_20170110232519225.jpg',
 'data/children/13_0_0_20170110232526929.jpg',
 'data/children/13_0_1_20170110232537879.jpg',
 'data/children/13_0_2_20170103201143159.jpg']

In [ ]:
random.seed(42)
random.shuffle(all_images)
all_images[:5]


['data/children/13_0_0_20170110232526929.jpg',
 'data/children/13_1_0_20170109203322570.jpg',
 'data/female adult/49_1_0_20170109220611995.jpg',
 'data/male adult/27_0_4_20170103235409988.jpg',
 'data/female adult/28_1_0_20170105165058893.jpg']

In [ ]:
import os
for impath in all_images:
  img = cv2.imread(impath)
  resized = cv2.resize(img, (HP_image_dim[0],HP_image_dim[1]) )
  imageData = img_to_array(resized)
  data.append(imageData)
  # extract label from filename (2nd last element) / \\ 
  label = impath.split(os.path.sep)[-2]
  labels.append(label)

In [ ]:
data = np.array(data, dtype='float' )/ 255.0
labels = np.array(labels)
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels

array([[1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0,

In [ ]:
from sklearn.model_selection import train_test_split
trainx,testx,trainy,testy = train_test_split(data, labels, test_size=0.25, random_state=42)


In [ ]:
trainx

array([[[[0.89019608, 0.97254902, 0.82352941],
         [0.9254902 , 0.98431373, 0.85882353],
         [0.94901961, 0.98431373, 0.86666667],
         ...,
         [0.94901961, 0.99607843, 0.90980392],
         [0.96470588, 0.98431373, 0.90980392],
         [0.92941176, 1.        , 0.90196078]],

        [[0.87058824, 0.95294118, 0.80392157],
         [0.90588235, 0.96470588, 0.83921569],
         [0.96078431, 0.99607843, 0.88235294],
         ...,
         [0.95294118, 1.        , 0.91372549],
         [0.96078431, 0.99607843, 0.89411765],
         [0.93333333, 0.99215686, 0.89019608]],

        [[0.90196078, 0.96470588, 0.83137255],
         [0.85882353, 0.96078431, 0.81960784],
         [0.94509804, 0.98823529, 0.85882353],
         ...,
         [0.94901961, 0.99607843, 0.90980392],
         [0.96078431, 0.99215686, 0.90196078],
         [0.92941176, 0.99215686, 0.89803922]],

        ...,

        [[0.09803922, 0.0745098 , 0.05490196],
         [0.1254902 , 0.08235294, 0.06666667]

In [ ]:
#Augmentation
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.2, 
                         height_shift_range=0.2, shear_range=0.2,
                         zoom_range=0.2, horizontal_flip=True,fill_mode='nearest')

In [ ]:
from keras import backend
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
#N= 5
HP_block1_conv_dim = 32
HP_block2_conv_dim = 64
HP_block3_conv_dim = 128
HP_block4_conv_dim = 256
HP_block5_dense_dim = 1024
HP_small_pattern = (3,3) # UNCOMPRESSED or 1-2 compression IMAGES
HP_large_pattern = (2,2) # 4 times compressed images from previous MP layers!!!
HP_dropout_little =0.25
HP_dropout_big = 0.50
# HP_epochs, batch_size-> are now problems of the developer USING this model. 

HP_img_dims = (96,96,3)


In [ ]:
class RacoonVGG:
  @staticmethod
  def build(height, width, depth, classes):
    # assume that we are on TF, but if something else is detected, switch the dimension
    input_shape = (height, width, depth)
    channel_dim = -1 # last position 
    if backend.image_data_format() == 'channels_first':
      input_shape = (depth, height, width)
      channel_dim = 1
    model = Sequential()
    
    # BLOCK1
    model.add(Conv2D(HP_block1_conv_dim,HP_small_pattern, padding='same',
                     input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_small_pattern))
    model.add(Dropout(HP_dropout_little))

    # COMPLEX BLOCK 2
    model.add(Conv2D(HP_block2_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(HP_block2_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_large_pattern))
    model.add(Dropout(HP_dropout_little))
    
    # COMPLEX BLOCK 3
    model.add(Conv2D(HP_block3_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(HP_block3_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_large_pattern))
    model.add(Dropout(HP_dropout_little))

    # COMPLEX BLOCK 4
    model.add(Conv2D(HP_block4_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(HP_block4_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_large_pattern))
    model.add(Dropout(HP_dropout_little))

    # BLOCK 5- Image Classification (OBJECT)
    model.add(Flatten())
    model.add(Dense(HP_block5_dense_dim))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(HP_dropout_big))
    model.add(Dense(classes))
    model.add(Activation('softmax'))
    return model


    # TF-> (h,w,d)
    # Theano-> (d,h,w)
    # GENERIC PROGRAM-> support BOTH the methods!!! 
    # if using KERAS-> because KERAS is just an API that is front facing an unknown
    # backend!

In [ ]:
from tensorflow.keras.optimizers import Adam
model = RacoonVGG.build(height=HP_img_dims[0], width=HP_img_dims[1], depth=HP_img_dims[2],classes = len(lb.classes_))
opt = Adam(lr=HP_init_lr, decay = HP_init_lr/ HP_epoch)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
len(trainx)

138

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto')

In [ ]:
history = model.fit(trainx, trainy, epochs=10, 
                    validation_data=(testx, testy),
                    )

Train on 138 samples, validate on 47 samples
Epoch 1/10
138/138 [==============================] - 8s 56ms/step - loss: 1.8128 - accuracy: 0.3478 - val_loss: 1.0915 - val_accuracy: 0.3830
Epoch 2/10
138/138 [==============================] - 7s 48ms/step - loss: 1.8317 - accuracy: 0.4348 - val_loss: 1.1801 - val_accuracy: 0.2766
Epoch 3/10
138/138 [==============================] - 7s 48ms/step - loss: 1.8928 - accuracy: 0.5000 - val_loss: 1.1505 - val_accuracy: 0.2766
Epoch 4/10
138/138 [==============================] - 7s 47ms/step - loss: 1.1493 - accuracy: 0.6087 - val_loss: 1.2074 - val_accuracy: 0.2766
Epoch 5/10
138/138 [==============================] - 7s 47ms/step - loss: 1.1655 - accuracy: 0.6232 - val_loss: 1.1119 - val_accuracy: 0.3830
Epoch 6/10
138/138 [==============================] - 7s 47ms/step - loss: 0.9570 - accuracy: 0.7174 - val_loss: 1.2322 - val_accuracy: 0.3404
Epoch 7/10
138/138 [==============================] - 7s 47ms/step - loss: 0.8186 - accuracy: 0.6